In [ ]:
# import essentials
import os
import shutil

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import keras
from keras.models import Sequential,load_model,model_from_json,Model
from keras import regularizers
from keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau
from keras.layers import Dense,Conv2D,Activation,Flatten,Dropout,GlobalAveragePooling2D,Lambda
from keras.engine.input_layer import Input
from keras.applications import *
# from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import *
from keras.optimizers import SGD,Adam
from keras.utils.vis_utils import plot_model
from keras.utils.np_utils import to_categorical
import keras.backend as K
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from PIL import Image
from matplotlib import pyplot
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import csv
import h5py

# trial series to track our iterations
trial_num = 20
data_change = True # if the split of data is changed or images are not generated yet
extract_feature = True # if the model features haven't been extracted

# data info and organizations
img_width,img_height = 224,224
batch_size = 8
epochs = 50
# no / partial / collapse: total 515 {0: 240, 1: 223, 2: 52}
nb_class_train = np.array([240, 220, 52]) # do validation split later in code
over_sample_train = np.array([1, 1, 1]) # make sure %batch_size = 0
nb_train_samples = np.dot(nb_class_train, over_sample_train.T)
print(nb_train_samples)
nb_test_samples = 129
num_classes = nb_class_train.shape[0]
channel = 3

# models used
models = {'ResNet50': [ResNet50, (224, 224), resnet50.preprocess_input], \
          'InceptionV3': [InceptionV3, (299, 299), inception_v3.preprocess_input], \
          'Xception': [Xception, (299, 299), xception.preprocess_input], \
          'VGG16': [VGG16, (224, 224), vgg16.preprocess_input], \
          'VGG19': [VGG19, (224, 224), vgg19.preprocess_input], \
          'InceptionResNetV2': [InceptionResNetV2, (299, 299), inception_resnet_v2.preprocess_input], \
          'MobileNet': [MobileNet, (224, 224), mobilenet.preprocess_input], \
          'MobileNetV2': [MobileNetV2, (224, 224), mobilenet.preprocess_input], \
#           'DenseNet121': [DenseNet121, (224, 224), densenet.preprocess_input], \
#           'DenseNet169': [DenseNet169, (224, 224), densenet.preprocess_input], \
#           'DenseNet201': [DenseNet201, (224, 224), densenet.preprocess_input], \
          'NASNetMobile': [NASNetMobile, (224, 224), nasnet.preprocess_input], \
          'NASNetLarge': [NASNetLarge, (331, 331), nasnet.preprocess_input]
         }
# models = {'VGG16': [VGG16, (299, 299), vgg16.preprocess_input]}

# path info
org_X_train_data_dir = './data_org/X_train.npy'
org_Y_train_data_dir = './data_org/Y_train.npy'
train_data_dir = './data_models/train'
test_data_dir = './data_models/test'

feature_path = './model_features'

best_model_weights_path = './merge_results/best_weights' + str(trial_num) + '.h5'
final_model_weights_path = './merge_results/final_weights' + str(trial_num) + '.h5'
model_save_path = './merge_results/model' + str(trial_num) + '.json'
learning_hist_path = './merge_results/learning_hist' + str(trial_num) + '.npy'
acc_plt_path1 = './merge_results/acc_plt1_' + str(trial_num) + '.png'
acc_plt_path2 = './merge_results/acc_plt2_' + str(trial_num) + '.png'
loss_plt_path1 = './merge_results/loss_plt1_' + str(trial_num) + '.png'
loss_plt_path2 = './merge_results/loss_plt2_' + str(trial_num) + '.png'
Y_predict_path = './merge_results/Y_predict' + str(trial_num) + '.csv'



In [ ]:
# generate temporary images for Keras' sequence
def process_data():
    # load data
    X_train = np.load(org_X_train_data_dir);
    Y_train = np.load(org_Y_train_data_dir);
    
    for c in range(num_classes):
        # make directory
        os.makedirs(train_data_dir+'/'+str(c))
        # over sampling training data
        index_c = np.where(Y_train == c)[0]
        X_train_c = X_train[index_c[0:nb_class_train[c]],:,:,:]
        X_train_new_c = np.tile(X_train_c,(int(over_sample_train[c]),1,1,1))
        extra_c = int((over_sample_train[c] - int(over_sample_train[c])) * nb_class_train[c])
        X_train_new_c = np.concatenate((X_train_new_c, X_train_c[0:extra_c,:,:,:]))
        # generate images
        for i in range(X_train_new_c.shape[0]):
            im = Image.fromarray(X_train_new_c[i, :, :, :])
            im.save(train_data_dir+'/'+str(c)+'/'+str(c)+'_'+str(i)+'.png')


In [ ]:
# re-generate images if data split is changed
if data_change:
    # delete all files and folders in directory
    shutil.rmtree(train_data_dir)
    # save new images
    process_data()

In [ ]:
# this function extracts the bottleneck features from a specified model
def write_gap(MODEL, model_str, image_size, lambda_func):
    width = image_size[0]
    height = image_size[1]
    input_tensor = Input((height, width, 3))
    x = input_tensor
    if lambda_func:
        x = Lambda(lambda_func)(x)
    
    # attach input and output layers to the pretrained model
    base_model = MODEL(input_tensor=x, weights='imagenet', include_top=False)
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))
    
    # generate data and predict
#     gen1 = ImageDataGenerator(rescale=1./255, # change here
#                                 shear_range=0.2,
#                                 zoom_range=0.2,
#                                 horizontal_flip=True)
    gen1 = ImageDataGenerator()
    gen2 = ImageDataGenerator()
    train_generator = gen1.flow_from_directory(train_data_dir, image_size, shuffle=False, batch_size=batch_size)
    test_generator = gen2.flow_from_directory(test_data_dir, image_size, shuffle=False, batch_size=1, class_mode=None)
    train = model.predict_generator(train_generator, train_generator.samples // batch_size)
    test = model.predict_generator(test_generator, test_generator.samples)
    print(train.shape)
    print(test.shape)
    print(train_generator.classes.shape)
#     print base_model.summary()
    # save features to file
    with h5py.File(feature_path+'/gap_%s.h5'%model_str) as h:
        h.create_dataset("train", data=train)
        h.create_dataset("test", data=test)
        h.create_dataset("label", data=train_generator.classes)

In [ ]:
# extract bottle neck feature from several pre-trained models
if extract_feature:
    # delete files in the directory
    shutil.rmtree(feature_path)
    os.makedirs(feature_path)
    # extract features
    for key, value in models.iteritems():
        write_gap(value[0], key, value[1], value[2])
        print "finish " + key
    

In [ ]:
# compute F1 score as a custom metric to return
def F1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
# read features of different models and shuffle
hist_info = {}
y_preds = {}
# loop through different models and fit model
for key, value in models.iteritems():
    filename = feature_path+'/gap_%s.h5'%key
    with h5py.File(filename, 'r') as h:
        X_train = np.array(h['train'])
        X_test = np.array(h['test'])
        y_train = np.array(h['label'])

    print(X_train.shape)
    print(X_test.shape)

    # shuffle
    np.random.seed(2018)
    X_train, y_train = shuffle(X_train, y_train)
    
    # build and train model
    y_train_cat = to_categorical(y_train, num_classes=num_classes)

    input_tensor = Input(X_train.shape[1:])
    print(X_train.shape[1:])
    x = Dropout(0.5)(input_tensor)
    x = Dense(num_classes, activation='softmax')(x)
    model = Model(input_tensor, x)
    print model.summary()

    sgd = SGD(lr=1e-4,momentum=0.9)
    model.compile(optimizer='adadelta',
                  loss='categorical_crossentropy',
                  metrics=['accuracy', F1])

    # early_stop = EarlyStopping(monitor='val_acc', patience=10, verbose=0, mode='auto')
    mcp_save = ModelCheckpoint(best_model_weights_path, save_best_only=True, monitor='val_F1', verbose=1, mode='max')
    reduce_lr_loss = ReduceLROnPlateau(monitor='val_F1', factor=0.9, patience=5, \
                                       min_lr=1e-4, verbose=1, mode='max') # change factor, patience and min_lr

    history = model.fit(X_train, y_train_cat, batch_size=nb_train_samples, epochs=100, validation_split=0.2, \
              callbacks = [mcp_save])
    
    # store history data in a dict
    hist_info[key] = history.history
    
    # save model for reference
    model_json = model.to_json()
    with open(model_save_path, "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights(final_model_weights_path)
    print("Saved model to disk")
    
    # load best model
    # load json and create model
    json_file = open(model_save_path, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights(best_model_weights_path)
    print("Loaded model from disk")
    
    # make prediction for current working model
    y_pred = loaded_model.predict(X_test, verbose=1)
    # save to a dict for later use
    y_preds[key] = y_pred

In [ ]:
# save learning history info to csv
np.save(learning_hist_path, hist_info)

# plot summary of learning
# matplotlib.style.use('default')
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 16}
plt.rcParams['axes.grid'] = True
matplotlib.rc('font', **font)
# # list all data in history
# print(hist_info.keys())
# summarize history for training accuracy
fig1 = plt.gcf()
for key, value in models.iteritems():
    hist_curr = hist_info[key]
    plt.plot(hist_curr['acc'], label=key)
#     plt.plot(hist_curr['val_acc'], label="test")
plt.title('Training accuracy, task 5')
plt.ylim(0.2, 0.9)
plt.xlim(0, 100)
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(bbox_to_anchor=(1.04, 0.5), loc='center left', prop={'size': 10}, ncol = 1)
plt.grid(color=(0.88,0.88,0.88), linestyle=':', linewidth=2)
plt.show()
fig1.savefig(acc_plt_path1, bbox_inches='tight', dpi=100)
# summarize history for validation accuracy
fig2 = plt.gcf()
for key, value in models.iteritems():
    hist_curr = hist_info[key]
    plt.plot(hist_curr['val_acc'], label=key)
plt.title('Validation accuracy, task 5')
plt.ylim(0.2, 0.9)
plt.xlim(0, 100)
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(bbox_to_anchor=(1.04, 0.5), loc='center left', prop={'size': 10}, ncol = 1)
plt.grid(color=(0.88,0.88,0.88), linestyle=':', linewidth=2)
plt.show()
fig2.savefig(acc_plt_path2, bbox_inches='tight', dpi=100)
# summarize history for trainging loss
fig3 = plt.gcf()
for key, value in models.iteritems():
    hist_curr = hist_info[key]
    plt.plot(hist_curr['loss'], label=key)
plt.title('Training loss, task 5')
plt.xlim(0, 100)
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(bbox_to_anchor=(1.04, 0.5), loc='center left', prop={'size': 10}, ncol = 1)
plt.grid(color=(0.88,0.88,0.88), linestyle=':', linewidth=2)
plt.show()
fig3.savefig(loss_plt_path1, bbox_inches='tight', dpi=100)
# summarize history for validation loss
fig4 = plt.gcf()
for key, value in models.iteritems():
    hist_curr = hist_info[key]
    plt.plot(hist_curr['val_loss'], label=key)
plt.title('Validation loss, task 5')
plt.xlim(0, 100)
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(bbox_to_anchor=(1.04, 0.5), loc='center left', prop={'size': 10}, ncol = 1)
plt.grid(color=(0.88,0.88,0.88), linestyle=':', linewidth=2)
plt.show()
fig4.savefig(loss_plt_path2, bbox_inches='tight', dpi=100)

In [ ]:
# print y_preds
print len(y_preds)
# print y_preds[next(iter(y_preds))]
k = y_preds[next(iter(y_preds))]
y_pred = np.zeros((nb_test_samples, num_classes))
# print y_pred
# y_pred = y_pred + y_preds[next(iter(y_preds))]
# print y_pred
for key, value in models.iteritems():
    curr_pred = y_preds[key]
    y_pred = y_pred + curr_pred / len(y_preds)
# print y_pred
# print np.sum(y_pred, axis = 1)

In [ ]:
# make prediction and save to csv
Y_predict = y_pred.argmax(axis=1)
Y_predict = Y_predict[:,np.newaxis]
# print(Y_predict)
print(Y_predict.shape)
index = np.arange(0, nb_test_samples)
out = np.concatenate((index[:, np.newaxis], Y_predict), axis = 1)
with open(Y_predict_path, "wb") as f:
    f.write(b'Index,Pred\n')
    np.savetxt(f, out.astype(int), fmt='%i', delimiter=",")